In [1]:
# 政府採購爬蟲
# url: https://web.pcc.gov.tw/prkms/prms-searchBulletionClient.do?root=tps

# request method: Post
import pymongo

client = pymongo.MongoClient("mongodb+srv://mimas9107:<password>@cluster0.amkt2.mongodb.net/myFirstDatabase?retryWrites=true&w=majority")
db = client.govbuy
col = db['訓練']
A1='機關名稱'
B1='標案名稱'
C1='捷標日期'
D1='網址'
E1='聯絡人'
F1='聯絡電話'
G1='預算金額'

In [2]:
import requests
from bs4 import BeautifulSoup
import time
import datetime
import random
header1={
    "User-Agent":"Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:88.0) Gecko/20100101 Firefox/88.0"
}
header2={
    "User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.149 Safari/537.36"
}
header3={
    "User-Agent":"Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.72 Safari/537.36"
}
headers_list=[header1,header2,header3]

headers=random.choice(headers_list)

In [3]:
url="https://web.pcc.gov.tw/prkms/prms-searchBulletionClient.do?root=tps"
form_data={
    "tmpQuerySentence": None,
    "timeRange": "110/1/1-110/12/31",
    "querySentence": "訓練",
    "tenderStatusType": "招標",
    "sortCol": "TENDER_NOTICE_DATE",
    "timeRangeTemp": "110/1/1-110/12/31",
    "sym": "on",
    "itemPerPage": "50"}

res=requests.post(url,form_data,headers=headers)
soup = BeautifulSoup(res.text)
time0 = time.time()
page=0
while page<65:
    page+=1
#    print('https://web.pcc.gov.tw/prkms/prms-searchBulletionClient.do'+soup.find_all('a',title='前往下一頁')[0]['href'])

    for case in soup.select('tbody')[0].select('tr'):
        
        time1=time.time()
        headers=random.choice(headers_list)
        res2=requests.get("https://web.pcc.gov.tw"+case.select('td')[3].a['href'],headers=headers)
        soup2 = BeautifulSoup(res2.text)
        time.sleep(random.randint(10,20))
#         print(soup2.select('table')[0].select('tr')[58].select('td')[0].text.replace('\t','').replace('\r','').replace('\n',''))
        
        i=40
        while i<61:
            i+=1
            if soup2.select('table')[0].select('tr')[i].select('th')[0].string == '聯絡人':
                print('聯絡人:',soup2.select('table')[0].select('tr')[i].select('td')[0].text.replace('\t','').replace('\r','').replace('\n',''))
                E2=soup2.select('table')[0].select('tr')[i].select('td')[0].text.replace('\t','').replace('\r','').replace('\n','')
            if soup2.select('table')[0].select('tr')[i].select('th')[0].string == '聯絡電話':
                print('聯絡電話:',soup2.select('table')[0].select('tr')[i].select('td')[0].text.replace('\t','').replace('\r','').replace('\n',''))
                F2=soup2.select('table')[0].select('tr')[i].select('td')[0].text.replace('\t','').replace('\r','').replace('\n','')
            if soup2.select('table')[0].select('tr')[i].select('th')[0].string == '預算金額':
                print('預算金額:',soup2.select('table')[0].select('tr')[i].select('td')[0].text.replace('\t','').replace('\r','').replace('\n',''))
                G2=soup2.select('table')[0].select('tr')[i].select('td')[0].text.replace('\t','').replace('\r','').replace('\n','')
        
        
        print('單位:',case.select('td')[2].string)
        A2=case.select('td')[2].string
        print('標案名稱:',case.select('td')[3].a.div.string)
        B2=case.select('td')[3].a.div.string
        print('截止日期:',case.select('td')[6].string)
        C2=case.select('td')[6].string
        # print("https://web.pcc.gov.tw"+case.select('td')[3].a['href'])
        D2="https://web.pcc.gov.tw"+case.select('td')[3].a['href']
        
        result=col.insert_one({A1:A2,B1:B2,C1:C2,D1:D2,E1:E2,F1:F2,G1:G2})
        print(result)
        time2=time.time()
        
        print('{:.2f}秒'.format(time2-time1))
        print('')
        
    print('****第{}頁爬取完成****'.format(page))
    
    # get and set the next page url
    url='https://web.pcc.gov.tw/prkms/prms-searchBulletionClient.do'+soup.find_all('a',title='前往下一頁')[0]['href']    
    
    headers=random.choice(headers_list)
    res=requests.post(url,form_data,headers=headers)
    soup = BeautifulSoup(res.text)
    time.sleep(random.randint(3,7))
    print(time2-time0)

聯絡人: 卓春成
聯絡電話: (07)8711130分機33
預算金額: 249,900元
單位: 高雄市立鳳林國民中學
標案名稱: 空手道訓練場燈光改善工程
截止日期: 110/07/14
12.290236949920654sec

聯絡人: 規格需求:體育組何志崇  採購:朱麗安
聯絡電話: (03)3392400分機518
預算金額: 1,666,667元
單位: 桃園市立青溪國民中學
標案名稱: 110年度充實體育器材設備(資本門)
截止日期: 110/07/15
15.52037239074707sec

聯絡人: 賓茂國小，承辦人：徐乙舜先生
聯絡電話: (089)771007分機13
預算金額: 300,000元
單位: 臺東縣政府
標案名稱: 賓茂國小--110年籃球訓練站體育館懸吊式籃架採購案
截止日期: 110/07/14
15.256779193878174sec

聯絡人: 李汶洙
聯絡電話: (07)7410111分機8022
預算金額: 863,310元
單位: 台灣電力股份有限公司鳳山區營業處
標案名稱: 裝備及訓練器材乙批
截止日期: 110/07/13
17.063910007476807sec

聯絡人: 鍾吉晟
聯絡電話: (02)27297668分機6517
預算金額: 200,000元
單位: 臺北市政府消防局
標案名稱: 110年度本局所屬各單位駐地大樓水塔及水池維護清潔案
截止日期: 110/07/15
14.074206590652466sec

聯絡人: 陳怡君
聯絡電話: (02)26095829分機134
預算金額: 3,803,613元
單位: 新北市立崇林國民中學
標案名稱: 崇林國中汙水下水道納管工程
截止日期: 110/07/13
13.18471074104309sec

聯絡人: 臺灣銀行採購部開標二科
聯絡電話: (02)23497673
預算金額: 20,920,000元
單位: 臺灣銀行股份有限公司
標案名稱: 香港分行洗錢防制系統名單掃描比對功能模組建置暨1年維護服務
截止日期: 110/07/23
13.120396614074707sec

聯絡人: 劉品伸
聯絡電話: (03)3255111分機3633
單位: 台灣中油股份有限公司煉製事業部桃園煉油廠
標案名稱: TESI點火控制系統

聯絡人: 黃紹豪
聯絡電話: (04)23335627分機408
預算金額: 8,872,500元
單位: 台灣電力股份有限公司台中供電區營運處
標案名稱: 備援系統主機及網路設備等一批
截止日期: 110/07/19
18.080531120300293sec

聯絡人: 黃全興
聯絡電話: (07)6912811分機3351
預算金額: 16,800,000元
單位: 台灣電力股份有限公司興達發電廠
標案名稱: 複3機氣渦輪機組Repowering保溫及排氣煙道升級改善工作
截止日期: 110/07/28
14.129507303237915sec

聯絡人: 楊錦裕
聯絡電話: (02)87712855
預算金額: 2,384,145,850元
單位: 內政部營建署
標案名稱: 現代化食品藥物國家級實驗大樓暨行政及訓練大樓興建工程
截止日期: 110/07/28
12.277740240097046sec

聯絡人: 林文雯
聯絡電話: (08)8782095分機34
預算金額: 1,260,300元
單位: 屏東縣立枋寮高級中學
標案名稱: 110年改善偏遠地區學校運動設備計畫-訓練器材
截止日期: 110/07/13
11.109959125518799sec

聯絡人: 林美伶
聯絡電話: (03)5326151分機6607
預算金額: 3,529,000元
單位: 國立臺灣大學醫學院附設醫院新竹臺大分院
標案名稱: 下肢肌力訓練機1台
截止日期: 110/07/13
19.1310555934906sec

聯絡人: 林發勳
聯絡電話: (03)4799595分機326557
預算金額: 600,000元
單位: 國防部軍醫局
標案名稱: 多人高壓艙維護保養
截止日期: 110/07/14
17.140363454818726sec

聯絡人: 馮彧
聯絡電話: (02)29173282分機342
預算金額: 42,050,000元
單位: 經濟部水利署臺北水源特定區管理局
標案名稱: 110-111年污水系統淨化槽及周邊附屬設施設置工程（北勢溪）
截止日期: 110/07/14
17.131324291229248sec

****第1頁爬取完成****
807.2519233226776
聯絡人: 呂慧君
聯絡電話: (05)2760474分機13

預算金額: 438,000元
單位: 彰化縣私立大慶高級商工職業學校
標案名稱: 110年度教育部體育署補助基層運動選手訓練站改善訓練環境及器材設備採購案
截止日期: 110/07/13
18.14130973815918sec

聯絡人: 關山國中，承辦人：林玫秀小姐
聯絡電話: (089)811114
預算金額: 627,830元
單位: 臺東縣政府
標案名稱: 關山國中--110年度基層運動選手訓練改善訓練環境及器材設備申請計畫案-體育器材購置
截止日期: 110/07/13
11.13640832901001sec

聯絡人: 許峻獻
聯絡電話: (03)5868019分機16
預算金額: 205,000元
單位: 新竹縣立石光國民中學
標案名稱: 110年度補助基層運動選手訓練站改善訓練環境及器材設備器材採購
截止日期: 110/07/15
15.517625093460083sec

聯絡人: 潘李美華
聯絡電話: (089)383629分機1007
預算金額: 281,000元
單位: 國立臺東大學附屬體育高級中學
標案名稱: 柔道訓練器材1批
截止日期: 110/07/12
13.249898433685303sec

聯絡人: 黃彥融
聯絡電話: (02)29173282分機341
預算金額: 42,083,588元
單位: 經濟部水利署臺北水源特定區管理局
標案名稱: 110-111年污水系統淨化槽及周邊附屬設施設置工程（南勢溪）
截止日期: 110/07/13
14.232635974884033sec

聯絡人: 黃純玲
聯絡電話: (02)23039978分機1102
預算金額: 150,000元
單位: 行政院農業委員會林業試驗所
標案名稱: 差分衛星定位儀1台
截止日期: 110/07/13
21.724502325057983sec

聯絡人: 羅金池
聯絡電話: (03)3286045分機1231
預算金額: 1,825,200元
單位: 台灣電力股份有限公司訓練所
標案名稱: 110年下半年學員餐盒訂購1式
截止日期: 110/07/16
11.373327732086182sec

聯絡人: 王信傑
聯絡電話: (089)570747分機207
預算金額: 369,000元
單位: 法務部矯正署東成技能訓練所
標案名稱: 門禁語

聯絡人: 陳宏明
聯絡電話: (049)2984006分機431
預算金額: 189,000元
單位: 臺灣菸酒股份有限公司埔里酒廠
標案名稱: COD加熱槽、多參數水質檢測儀、可見光分光光度計等3項儀器各1台
截止日期: 110/07/13
20.2595694065094sec

聯絡人: 開標二科
聯絡電話: (02)23497673
預算金額: 8,385,000元
單位: 臺灣銀行股份有限公司
標案名稱: 防毒軟體管理系統設備汰換暨4年維護
截止日期: 110/07/12
15.137350082397461sec

聯絡人: 劉果
聯絡電話: (08)7663408
預算金額: 539,000元
單位: 國防部空軍司令部
標案名稱: 航空噪音監測系統設備維護案(修訂後)
截止日期: 110/07/12
16.07333016395569sec

聯絡人: 包至玉
聯絡電話: (049)2231150分機2238
預算金額: 6,898,500元
單位: 衛生福利部南投醫院
標案名稱: 正子電腦斷層描PET-CT代檢業務採購案
截止日期: 110/07/13
11.960332155227661sec

聯絡人: 林芙如
聯絡電話: (049)2770075分機468
預算金額: 8,190,000元
單位: 台灣電力股份有限公司大觀發電廠
標案名稱: 110年度大觀二廠#1機大修及機械組設備檢修勞務性工作
截止日期: 110/07/15
15.940484523773193sec

聯絡人: 李小姐
聯絡電話: (049)2781077
預算金額: 45,000,000元
單位: 國防部軍備局生產製造中心
標案名稱: 配電箱等5項維修
截止日期: 110/07/13
18.97593355178833sec

聯絡人: 陳韋君
聯絡電話: (089)326121分機791
預算金額: 7,367,850元
單位: 台灣自來水股份有限公司第十區管理處
標案名稱: 大武系統-和平500m3配水池新建工程
截止日期: 110/07/14
15.970370769500732sec

聯絡人: 田先生
聯絡電話: (07)5851440分機1176
預算金額: 2,731,300元
單位: 國防部海軍司令部
標案名稱: 示波器組等2項
截止日期: 110/07/

聯絡人: 魏組長
聯絡電話: (03)5316605分機123
預算金額: 268,000元
單位: 新竹市立光華國民中學
標案名稱: 110年度基層運動選手訓練站辦理改善訓練環境及器材設備
截止日期: 110/07/09
15.52230453491211sec

聯絡人: 許祥瑞
聯絡電話: (03)5690772分機503
預算金額: 523,000元
單位: 新竹縣立湖口高級中學
標案名稱: 110年度基層選手訓練招標案田徑器材
截止日期: 110/07/08
12.966877937316895sec

聯絡人: 張曜麟(程序);黃婉惠(需求)
聯絡電話: (07)8210171分機1606;1211
預算金額: 8,400,000元
單位: 勞動部勞動力發展署高屏澎東分署
標案名稱: 110年度青年職業訓練專班計畫
截止日期: 110/07/14
19.148897171020508sec

聯絡人: 徐嘉宏
聯絡電話: (03)5802204分機513
預算金額: 800,000元
單位: 新竹縣北埔鄉公所
標案名稱: 「2021北埔膨風路跑活動」委託專業服務案
截止日期: 110/07/07
20.936114072799683sec

聯絡人: 林忠賢
聯絡電話: (089)230655
預算金額: 4,349,100元
單位: 法務部矯正署岩灣技能訓練所
標案名稱: 110年下半年度法務部矯正署所屬臺東2區矯正機關聯合招標收容人副食品-豬牛肉品類採購案
截止日期: 110/07/05
17.065542936325073sec

聯絡人: 規格請洽：張先生07-5861077；採購請洽：吳小姐07-5860192
聯絡電話: (07)5861092
預算金額: 1,400,000元
單位: 國家運動訓練中心
標案名稱: 110年度宿舍內部購置除濕機設備1批案
截止日期: 110/07/13
19.337228536605835sec

聯絡人: 陳竹士
聯絡電話: (07)6117101分機302
預算金額: 800,000元
單位: 交通部公路總局公路人員訓練所
標案名稱: 三輪大型重型機車身心障礙者考驗教練車1輛
截止日期: 110/07/05
11.975356340408325sec

聯絡人: 吳瑞香
聯絡電話: (03)4917491

預算金額: 191,000元
單位: 衛生福利部疾病管制署
標案名稱: 110年使用認證標誌之微量吸管校正
截止日期: 110/07/06
16.15373396873474sec

聯絡人: 林明緯
聯絡電話: (03)3321224分機605
預算金額: 420,000元
單位: 國軍退除役官兵輔導委員會退除役官兵職業訓練中心
標案名稱: 110年宿舍寢具採購案
截止日期: 110/07/05
13.000032663345337sec

聯絡人: 劉芳妮
聯絡電話: (02)25553000分機2126
預算金額: 398,000元
單位: 臺北市立聯合醫院
標案名稱: 步行訓練平行桿
截止日期: 110/07/06
10.987619161605835sec

聯絡人: 龔文賓
聯絡電話: (06)2664911分機1303
預算金額: 150,000元
單位: 嘉藥學校財團法人嘉南藥理大學
標案名稱: 自由車訓練用耗材
截止日期: 110/07/07
19.26883363723755sec

聯絡人: 洪秀琴
聯絡電話: (04)22391647分機8311
預算金額: 198,000元
單位: 中臺科技大學
標案名稱: 互動認知訓練板
截止日期: 110/07/05
18.929454803466797sec

聯絡人: 知本國中，承辦人：黃麗錦小姐
聯絡電話: (089)512203分機230
預算金額: 130,000元
單位: 臺東縣政府
標案名稱: 知本國中--110年基層運動選手訓練站訓練器材設備採購案
截止日期: 110/07/05
11.982924938201904sec

聯絡人: 陳佑辰
聯絡電話: (07)7835011分機125
預算金額: 989,000元
單位: 高雄市政府勞工局訓練就業中心
標案名稱: 110年即測即評技能檢定學科電腦教室設備採購契約
截止日期: 110/07/05
17.98875379562378sec

聯絡人: 潘宥亦(採購).吳展誼(規格)
聯絡電話: (03)8462119分機3206.3309
預算金額: 900,000元
單位: 花蓮縣消防局
標案名稱: 花蓮縣消防局110年空拍機(含紅外線熱顯像儀)
截止日期: 110/07/06
19.0139901638031sec

聯絡人: 黃唯銘

KeyboardInterrupt: 